In [14]:
import sys
import os

# Dynamically add the parent directory to the Python path
module_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [15]:
!pip install mpmath

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
from utils.physical_properties import sound_speed
from models.breathing_model import calculate_breathing_ts
from models.thuraisingham_model import calculate_thuraisingham_ts
from models.modal_solution import calculate_modal_ts
from models.medwin_clay_model import calculate_medwin_clay_ts
from utils.SeaEcho_water_bubble import seawater, air_bubble
import ipywidgets as widgets
from IPython.display import display

In [17]:
# Define the available models
models = {
    "Breathing Model": "calculate_breathing_ts",
    "Thuraisingham Model": "calculate_thuraisingham_ts",
    "Modal Solution": "calculate_modal_ts",
    "Medwin-Clay Model": "calculate_medwin_clay_ts",
}

# Create a dropdown widget
model_dropdown = widgets.Dropdown(
    options=models,
    value="calculate_breathing_ts",  # Default selection
    description="Scattering Model:",
    style={'description_width': 'initial'}
)

# Display the dropdown
display(model_dropdown)

# You can access the selected value like this
def on_model_change(change):
    print(f"Selected Model: {change['new']}")

# Attach a listener to detect when the selection changes
model_dropdown.observe(on_model_change, names='value')


Dropdown(description='Scattering Model:', options={'Breathing Model': 'calculate_breathing_ts', 'Thuraisingham…

In [18]:
# Create an Output widget to display results dynamically
output = widgets.Output()

# Create input widgets for physical properties with both slider and text input
temperature_slider = widgets.FloatSlider(
    value=25.0,
    min=-10.0,
    max=40.0,
    step=0.1,
    description='Temperature (°C):',
    style={'description_width': 'initial'},
    continuous_update=False
)

temperature_text = widgets.FloatText(
    value=25.0,
    description='',
    style={'description_width': 'initial'}
)

# Link slider and text input
widgets.jslink((temperature_slider, 'value'), (temperature_text, 'value'))

salinity_slider = widgets.FloatSlider(
    value=35.0,
    min=0.0,
    max=50.0,
    step=0.1,
    description='Salinity (PSU):',
    style={'description_width': 'initial'},
    continuous_update=False
)

salinity_text = widgets.FloatText(
    value=35.0,
    description='',
    style={'description_width': 'initial'}
)

widgets.jslink((salinity_slider, 'value'), (salinity_text, 'value'))

depth_slider = widgets.FloatSlider(
    value=1000.0,
    min=0.0,
    max=11000.0,
    step=10.0,
    description='Depth (m):',
    style={'description_width': 'initial'},
    continuous_update=False
)

depth_text = widgets.FloatText(
    value=1000.0,
    description='',
    style={'description_width': 'initial'}
)

widgets.jslink((depth_slider, 'value'), (depth_text, 'value'))

# Arrange sliders and text inputs in a horizontal layout
temperature_box = widgets.HBox([temperature_slider, temperature_text])
salinity_box = widgets.HBox([salinity_slider, salinity_text])
depth_box = widgets.HBox([depth_slider, depth_text])

# Global dictionary to store inputs for further use
physical_properties = {}

# Access and validate the selected values, then calculate sound speed
def calculate_physical_properties():
    temperature = temperature_slider.value
    salinity = salinity_slider.value
    depth = depth_slider.value
    
    # Validation
    with output:
        output.clear_output()  # Clear previous output
        if temperature < -10 or temperature > 40:
            print("Error: Temperature must be between -10°C and 40°C.")
            return
        if salinity < 0 or salinity > 50:
            print("Error: Salinity must be between 0 PSU and 50 PSU.")
            return
        if depth < 0 or depth > 11000:
            print("Error: Depth must be between 0 m and 11,000 m.")
            return

        # Initialize the seawater object
        water = seawater(temperature=temperature, depth=depth, salinity=salinity)

        # Retrieve sound speed, density, and pressure
        c = water.sound_speed()  # Sound speed
        rho = water.density()    # Density
        P = water.pressure()     # Pressure

        # Save properties for further use
        physical_properties['temperature'] = temperature
        physical_properties['salinity'] = salinity
        physical_properties['depth'] = depth
        physical_properties['sound_speed'] = c
        physical_properties['density'] = rho
        physical_properties['pressure'] = P

        # Display valid inputs and calculated properties
        print(f"Sound Speed: {c:.2f} m/s")
        print(f"Density: {rho:.2f} kg/m³")
        print(f"Pressure: {P:.2f} Pa")

# Button to calculate properties
calculate_button = widgets.Button(description="Calculate Properties")
calculate_button.on_click(lambda b: calculate_physical_properties())

# Group all elements into a vertical layout for better organization
ui = widgets.VBox([
    widgets.Label("Set Physical Properties"),
    temperature_box,
    salinity_box,
    depth_box,
    calculate_button,
    output
])

# Display the UI
display(ui)


In [19]:
from utils.SeaEcho_water_bubble import seawater, air_bubble

# Create input fields for frequency range and points
min_freq_input = widgets.FloatText(
    value=10.0,
    description="Min Frequency (Hz):",
    style={"description_width": "initial"}
)
max_freq_input = widgets.FloatText(
    value=5000.0,
    description="Max Frequency (Hz):",
    style={"description_width": "initial"}
)
num_points_input = widgets.IntText(
    value=500,
    description="Number of Points:",
    style={"description_width": "initial"}
)

# Group frequency inputs in a single box for a cleaner UI
frequency_inputs = widgets.VBox([min_freq_input, max_freq_input, num_points_input])
display(frequency_inputs)

# Function to execute the selected scattering model
def execute_model(*args):
    # Get the selected scattering model
    selected_model = model_dropdown.value

    # Retrieve physical property inputs
    temperature = temperature_slider.value
    salinity = salinity_slider.value
    depth = depth_slider.value

    # Retrieve user-input frequency range
    min_freq = min_freq_input.value
    max_freq = max_freq_input.value
    num_points = num_points_input.value

    # Validate frequency inputs
    with output:
        output.clear_output()
        if min_freq >= max_freq:
            print("Error: Min Frequency must be less than Max Frequency.")
            return
        if num_points <= 0:
            print("Error: Number of Points must be positive.")
            return

        # Initialize seawater object with physical properties
        water = seawater(temperature=temperature, depth=depth, salinity=salinity)
        c = water.sound_speed()
        print(f"Calculated Sound Speed: {c:.2f} m/s")

        # Define frequency range based on user input (convert to kHz)
        frequencies = np.linspace(min_freq, max_freq, num_points) / 1000  # Convert Hz → kHz

        # Initialize bubble object (default example values for diameter)
        bubble_diameter = 0.001  # Example bubble diameter in meters
        bubble = air_bubble(water, temperature, depth, salinity, bubble_diameter)

        # Call the selected model's function
        if selected_model == "calculate_breathing_ts":
            results = [calculate_breathing_ts(f, c, water, bubble) for f in frequencies]
            model_name = "Breathing Model"
        elif selected_model == "calculate_thuraisingham_ts":
            results = [calculate_thuraisingham_ts(f, c, water, bubble) for f in frequencies]
            model_name = "Thuraisingham Model"
        elif selected_model == "calculate_modal_ts":
            results = [calculate_modal_ts(f, c, water, bubble) for f in frequencies]
            model_name = "Modal Solution"
        elif selected_model == "calculate_medwin_clay_ts":
            results = [calculate_medwin_clay_ts(f, c, water, bubble) for f in frequencies]
            model_name = "Medwin-Clay Model"
        else:
            print("Invalid model selected!")
            return

        # Plot the results
        plt.figure(figsize=(8, 6))
        plt.plot(frequencies * 1000, results, label=f"{model_name}")  # Convert back to Hz for plotting
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("Target Strength (dB)")
        plt.title(f"{model_name} Target Strength vs Frequency")
        plt.grid(True)
        plt.legend()
        plt.show()

# Button to execute the model
execute_button = widgets.Button(description="Run Model")
execute_button.on_click(execute_model)
display(execute_button)

Button(description='Run Model', style=ButtonStyle())